In [ ]:
#The worklow of this code contains several parts:
# Data Cleaning, Calculating Medians (more resistable for outliers)and Distributions, and Machine Learning Models

In [ ]:
# PART I: DATA CLEANING

In [ ]:
# PART II: CALCULATING AVERAGES AND DISTRIBUTIONS

In [9]:
# MACHINE LEARNING MODELS USING CLEAN DATA
# Both Linear Regression and Deep Learning models will be used in this part

In [10]:
# Install Spark and Java
import os
# Set Environment Variables for Spark
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

#Import Pandas
import pandas as pd

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Java  and Spark Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Initialize PySpark
import findspark
findspark.init()

# Import PySpark DataFrame functions (round, col, sum etc.) with shorter alias
from pyspark.sql import functions as F

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [11]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").config("spark.driver.memory", "2g").getOrCreate()

In [12]:
# Upload the file to Google Colab
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload the file

Saving movie_metadata.csv to movie_metadata (1).csv


In [13]:
# Read the uploaded CSV file into a DataFrame
movie_data = pd.read_csv('movie_metadata.csv')

# 3. Display the first few rows of the DataFrame
movie_data.head()


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [17]:
# Get the number of rows in the date (data size)
num_rows = movie_data.shape[0]

# Print the result
print(f"Number of rows in the dataset: {num_rows}")

Number of rows in the dataset: 5043


In [18]:
# Verify success of data clean up: Check for missing values in the 'updated_data' DataFrame
missing_values = movie_data.isnull().sum()

# Display the result
print(missing_values)

color                         19
director_name                104
num_critic_for_reviews        50
duration                      15
director_facebook_likes      104
actor_3_facebook_likes        23
actor_2_name                  13
actor_1_facebook_likes         7
gross                        884
genres                         0
actor_1_name                   7
movie_title                    0
num_voted_users                0
cast_total_facebook_likes      0
actor_3_name                  23
facenumber_in_poster          13
plot_keywords                153
movie_imdb_link                0
num_user_for_reviews          21
language                      14
country                        5
content_rating               303
budget                       492
title_year                   108
actor_2_facebook_likes        13
imdb_score                     0
aspect_ratio                 329
movie_facebook_likes           0
dtype: int64


In [19]:
# Further look at our data: Determine the number of unique values in each column.
unique_values = movie_data.nunique()
print(unique_values)

color                           2
director_name                2398
num_critic_for_reviews        528
duration                      191
director_facebook_likes       435
actor_3_facebook_likes        906
actor_2_name                 3032
actor_1_facebook_likes        878
gross                        4035
genres                        914
actor_1_name                 2097
movie_title                  4917
num_voted_users              4826
cast_total_facebook_likes    3978
actor_3_name                 3521
facenumber_in_poster           19
plot_keywords                4760
movie_imdb_link              4919
num_user_for_reviews          954
language                       46
country                        65
content_rating                 18
budget                        439
title_year                     91
actor_2_facebook_likes        917
imdb_score                     78
aspect_ratio                   22
movie_facebook_likes          876
dtype: int64


In [22]:
# Check how proportionally data distributed in 'gross'and 'budget' columns (to chech how balanced our data)
gross_type_counts = movie_data['gross'].value_counts()
budget_type_counts = movie_data['budget'].value_counts()
# Display the results
print(gross_type_counts)
print(budget_type_counts)


gross
47000000.0     3
34964818.0     3
5773519.0      3
3000000.0      3
177343675.0    3
              ..
39026186.0     1
33422806.0     1
33423521.0     1
32519322.0     1
85222.0        1
Name: count, Length: 4035, dtype: int64
budget
20000000.0    174
15000000.0    143
25000000.0    142
30000000.0    141
10000000.0    135
             ... 
10818775.0      1
13800000.0      1
12305523.0      1
12600000.0      1
1100.0          1
Name: count, Length: 439, dtype: int64


In [23]:
# Convert the Pandas DataFrame to a PySpark DataFrame
spark_movie_data = spark.createDataFrame(movie_data)

# Create a temporary view of the PySpark DataFrame
spark_movie_data.createOrReplaceTempView('movies')

# Get the data types of the columns for the PySpark DataFrame
spark_movie_data.printSchema()

# Run SQL queries on the DataFrame, use spark.sql
spark.sql("SELECT * FROM movies LIMIT 3").show()

root
 |-- color: string (nullable = true)
 |-- director_name: string (nullable = true)
 |-- num_critic_for_reviews: double (nullable = true)
 |-- duration: double (nullable = true)
 |-- director_facebook_likes: double (nullable = true)
 |-- actor_3_facebook_likes: double (nullable = true)
 |-- actor_2_name: string (nullable = true)
 |-- actor_1_facebook_likes: double (nullable = true)
 |-- gross: double (nullable = true)
 |-- genres: string (nullable = true)
 |-- actor_1_name: string (nullable = true)
 |-- movie_title: string (nullable = true)
 |-- num_voted_users: long (nullable = true)
 |-- cast_total_facebook_likes: long (nullable = true)
 |-- actor_3_name: string (nullable = true)
 |-- facenumber_in_poster: double (nullable = true)
 |-- plot_keywords: string (nullable = true)
 |-- movie_imdb_link: string (nullable = true)
 |-- num_user_for_reviews: double (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- content_rating: string (nu

In [25]:
# Convert the 'duration', 'gross', and 'budget' columns from double to integer
spark_movie_data = spark_movie_data \
    .withColumn("duration", F.col("duration").cast("int")) \
    .withColumn("gross", F.col("gross").cast("int")) \
    .withColumn("budget", F.col("budget").cast("int"))

# Verify the schema after conversion
spark_movie_data.printSchema()
# Run a SQL query to verify the changes
spark_movie_data.createOrReplaceTempView('movies')
spark.sql("SELECT duration, gross, budget FROM movies LIMIT 3").show()

root
 |-- color: string (nullable = true)
 |-- director_name: string (nullable = true)
 |-- num_critic_for_reviews: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- director_facebook_likes: double (nullable = true)
 |-- actor_3_facebook_likes: double (nullable = true)
 |-- actor_2_name: string (nullable = true)
 |-- actor_1_facebook_likes: double (nullable = true)
 |-- gross: integer (nullable = true)
 |-- genres: string (nullable = true)
 |-- actor_1_name: string (nullable = true)
 |-- movie_title: string (nullable = true)
 |-- num_voted_users: long (nullable = true)
 |-- cast_total_facebook_likes: long (nullable = true)
 |-- actor_3_name: string (nullable = true)
 |-- facenumber_in_poster: double (nullable = true)
 |-- plot_keywords: string (nullable = true)
 |-- movie_imdb_link: string (nullable = true)
 |-- num_user_for_reviews: double (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- content_rating: string (

In [ ]:
## Consider creating beans for Linear regression and Deep learning

In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table movies")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('movies')

True

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table movies")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("movies")

False